# Grundlagen der KI - Gruppenprojekte


Für die Projekte stellen wir eine MongoDB Datenbank, die wir aktiv in der Forschung nutzen, zur Verfügung. Die Datenbank beinhaltet Informationen über Versionskontrollsysteme, Issuetracker, und Mailinglisten von Open Source Projekten. [Die Dokumentation der Datenbank finden Sie Online](https://smartshark2.informatik.uni-goettingen.de/documentation/).

## Die Aufgabe

Ihre Aufgabe ist es, einen automatisierten Ansatz zu entwickeln, welcher vorhersagt, ob ein Issue innerhalb von 6 Monaten behoben wird oder nicht. Darüber hinaus, sollte das Modell (oder auch ein zweites Modell) in der Lage sein, die erwartete Zeit für die Behebung und damit Schließung des Issues vorherzusagen.
Sie können dazu beliebige Informationen aus der Datenbank nutzen, wie z.B. Commits, andere Issues, Mails und Änderungen in den Dateien.
Sie müssen diese Problemstellung als Analyseproblem fassen um dann Modelle zur Lösung aufstellen zu können. Hierzu müssen Sie geeignete Analysemethoden auswählen und anschließend die Qualität der Ergebnisse evaluieren. 

## Präsentationen

**Alle Gruppen müssen ihre Ergebnisse in der letzten Vorlesung am 09.02. um 16:15 Uhr vorstellen.** Hierzu muss jede Gruppe eine kurze Präsentation halten. In dieser Präsentation sollten Sie kurz beschreiben wie Sie vorgegangen sind, wie Sie Features erstellt haben, welche Modelle Sie benutzt haben und was ihre wichtigsten Ergebnisse sind. Die Zeit für die Präsentation beträgt 9 (1) Minuten.

Im Anschluss an die Präsentation stimmen die Anwesenden über das beste Projekt ab. Jede Gruppe stimmt für das beste Projekt (3 Punkte), das zweitbeste (2 Punkte) und das drittbeste (1 Punkt). Das Projekt mit den meisten Punkten gewinnt einen Preis.

Die Präsentation wird in Präsenz stattfinden, wenn die aktuellen Regeln es zulassen (Prof. Herbold wird dabei per Video zugeschaltet sein; Raum wird noch bekannt gegeben). Da wir nur begrenzt Zeit haben, müssen alle Präsentationen vorab geschickt werden. Wir kombinieren diese dann in einer PDF um einen einfachen Wechsel zwischen den Präsentation der Gruppen zu erreichen. Die Mitglieder der Gruppe werden dann in der Reihenfolge des Gruppennames gebeten ihre Ergebnisse vorzustellen, d.h. wir beginnen mit Gruppe 1 und enden mit Gruppe 6.


## Abgabe der Präsentationen

Damit die Präsentation reibungslos ablaufen, müssen Sie ihre Präsentation im Vorfeld einsenden. **Schicken Sie Ihre Präsentationen bitte bis spätestens zum 08.02.23, 16 Uhr an Clemens Dierks via Email (clemens.dierks@tu-clausthal.de)**. Die Präsentation muss als PDF vorliegen.


## Mindestkriterien

Die folgenden Kriterien muss ein Projekt mindestens erfüllen, damit die Gruppenmitglieder an der Prüfung teilnehmen können.
- Es muss ein Modell für die automatisierte Vorhersage, ob ein Issue innerhalb von 6 Monaten gelöst wird, vorhanden sein.
- Es muss ein Modell für die automatisierte Vorhersage der estimated-time-to-fix von Issues vorhanden sein.
- Die Qualität des Modells muss bewertet werden.
- Sie müssen eine Empfehlung geben, ob und ggf. unter welchen Umständen, das Modell eingesetzt werden kann basierend auf der Evaluation der Qualität des Modells.
- Aufgrund Ihrer Empfehlung können Sie noch Verbesserungsmöglichkeiten vorstellen.
- Die Ergebnisse müssen in der Vorlesung am 09.02.23 vorgestellt werden.

## Datenbankzugriff

Unterhalb finden Sie ein Beispiel, wie Sie auf die Datenbank zugreifen. Die Datenbank befindet sich hinter einer Firewall und kann nur aus dem Netz der TU Clausthal erreicht werden. Wenn Sie Probleme haben die Datenbank zu erreichen, können Sie von jedem beliebigen Netzwerk aus in das Netz der TU Clausthal gelangen, in dem Sie sich mit dem [VPN](https://doku.tu-clausthal.de/doku.php?id=netzwerk_und_internet:vpn:start) verbinden.

**WARNUNG:
Wir benutzen die Datenbank aktiv in der Forschung. Es kann daher sein, das teilweise eine sehr hohe Last auf der Datenbank ist. Die Datenbank ist sehr groß und beinhaltet derzeit 4.8 Terabyte an Daten. Sie sollten daher nicht einfach starten Anfragen zu schreiben, sondern sich genau überlegen auf welche Informationen Sie zugreifen wollen. Wenn Sie zum Beispiel eine Anfrage stellen, in der Sie die komplette ```commit``` Collection herunterladen möchten, müssen Sie hierfür 158 Gigabyte an Daten runterladen. **

Sie können die Bibliothek [pycoSHARK](https://github.com/smartshark/pycoshark) zum Zugriff auf die Datenbank nutzen. Der pycoSHARK stellt einen ORM-Layer basierend auf der Bibliothek ```mongoengine``` zur Verfügung. Hierdurch können Sie die Datenbanktabellen wie Objekte behandeln. Alternativ können Sie die native MongoDB API der ```pymongo``` Bibliothek benutzen. 

Der Quelltext unten beinhaltet Ihre Zugangsdaten in zeigt wie man mit dem pycoSHARK auf die Datenbank zugreift.

Interesting features: No. of commits of the issue creator,age of the branch 

In [ ]:
# Installation des pycoSHARKs
import sys
# !{sys.executable} -m pip install pycoshark

In [45]:
from mongoengine import connect, disconnect
from pycoshark.mongomodels import File, FileAction, Commit, Project, VCSSystem, IssueSystem,Hunk, Refactoring, IssueSystem, Issue, IssueComment, MailingList, Message
from pycoshark.utils import create_mongodb_uri_string
import pandas as pd
import pandas_profiling as pp #best to be installed with anaconda since pip sometimes makes problems with pandas_profiling
from sklearn import tree
from sklearn.model_selection import train_test_split # Import train_test_split function
from sklearn import metrics #Import scikit-learn metrics module for accuracy calculation
# Database credentials
user = 'grundlagenki2021'
password = '38BTGx71d'
host = '134.76.81.151'
port = '27017'
authentication_db = 'smartshark'
database = "smartshark"
ssl_enabled = None

# Establish connection
uri = create_mongodb_uri_string(user, password, host, port, authentication_db, ssl_enabled)
connect(database, host=uri)

# Fetch project id and issue tracking system system id for the 'commons-math' project
# The only() decides the data that is actually retrieved from the MongoDB. 
# get() only works, when you query results in exactly one document, otherwise you have to use a loop to access the data
# an example with a loop is at the bottom of this example
#project = Project.objects(name='commons-math').only('id').get()
#issue_system = IssueSystem.objects(project_id=project.id).only('id','url').get()

for items in Project.objects:
    print (items.name)
def getreport(projectnames):
    projectdf = []
    for items in (projectnames):
        
    # We first need the the project from the database
        project = Project.objects(name=items).get()
        issue_tracker = IssueSystem.objects(project_id=project.id).get()
        dfin=pd.read_json(Issue.objects(issue_system_id=issue_tracker.id).to_json())
        extid=issue_tracker.url.rsplit('=', 1)[1] # obtain the correct external_id for each repository to sort out invalid issues or logs
        dfin=dfin[dfin.external_id.str.startswith(extid)]#print(projectdf)
        print('read'+str(items))
        projectdf.append(dfin)
        print(len(projectdf))
    #print(projectdf[0])    
    df1 = pd.concat(projectdf)   
    print(df1.shape)    
    df=df1
    
    df['created_at']=pd.json_normalize(df['created_at'])
    df['updated_at']=pd.json_normalize(df['updated_at'])
    df['timetofix']=(df.updated_at-df.created_at)/(100*60*60*24*30.41)
    df['status'] = df['status'].replace('Closed', 1)
    df['status'] = df['status'].replace('In Progress', 0)
    df['status'] = df['status'].replace('Open', 0)
    df['status'] = df['status'].replace('Resolved', 1)
    df['status'] = df['status'].replace('Reopened', 0)
    df['status'] = df['status'].replace('Patch Available', 0)
    #df=df[['external_id', 'creator_id', 'issue_type', 'priority', 'status' , 'timetofix']]
    df=df[[ 'creator_id','issue_type', 'priority', 'status' , 'timetofix']]
    df = pd.get_dummies(data=df, columns=['priority','issue_type'])
    
    return(df)

commons-math
oisafe
ant-ivy
junit
kafka
zeppelin
mahout
struts
opennlp
pig
calcite
cayenne
falcon
kylin
nutch
parquet-mr
tika
wss4j
archiva
deltaspike
jspwiki
xerces2-j
tez
systemml
storm
lens
accumulo
commons-lang
james
pdfbox
flume
nifi
knox
derby
commons-collections
commons-beanutils
commons-codec
commons-compress
commons-configuration
commons-digester
commons-imaging
commons-io
commons-jcs
commons-jexl
commons-rdf
commons-scxml
commons-validator
commons-vfs
giraph
manifoldcf
ranger
eagle
commons-bcel
commons-dbcp
zookeeper
phoenix
gora
santuario-java
helix
jena
httpcomponents-client
commons-net
httpcomponents-core
streams
samza
roller
mina-sshd
bigtop
jackrabbit
tomcat
rhino
oozie
avro
activemq
fop
maven
airavata
openjpa
AutoUpdater.NET
directory-fortress-core
eShopOnWeb
EquinoxProject
BlazorWithIdentity
ruler
quick-picture-viewer
Mynt
SuperSafeBank
Hunt
Windows-appsample-networkhelper
Windows-appsample-customers-orders-database
Miniblog.Core
coolstore-microservices
clean-architect

Copied from https://github.com/smartshark/usage-examples/blob/main/Example-Notebook.ipynb:

In [2]:
print(Project.objects[5].name)

zeppelin


In [111]:
projects=['calcite','kylin','parquet-mr','cayenne','falcon','uima-ducc',]
df5=getreport(projects)
#print(df5)
df5.shape
df5.head()


readcalcite
1
readkylin
2
readparquet-mr
3
readcayenne
4
readfalcon
5
readuima-ducc
6
(15746, 25)


,creator_id,status,timetofix,priority_Blocker,priority_Critical,priority_Major,priority_Minor,priority_Trivial,issue_type_Brainstorming,issue_type_Bug,...,issue_type_Improvement,issue_type_New Feature,issue_type_Question,issue_type_Story,issue_type_Sub-task,issue_type_Suitable Name Search,issue_type_Task,issue_type_Technical task,issue_type_Test,issue_type_Wish
0,{'$oid': '5b3b320030a71b06bc6c8ea1'},1,17.779244,0,0,1,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
1,{'$oid': '5b3b320030a71b06bc6c8ea1'},1,17.779873,0,0,1,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
2,{'$oid': '5b3b320630a71b06bc6cc667'},1,17.725177,0,0,1,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
3,{'$oid': '5b3b302130a71b06bc5042e9'},1,22.499207,0,0,1,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
4,{'$oid': '5b35273530a71b06bc5391ba'},1,2.491320,0,0,1,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0


In [ ]:
pp.ProfileReport(df5, title="Pandas Profiling Report")

In [104]:
df5.head()

,creator_id,status,timetofix,priority_Blocker,priority_Critical,priority_Major,priority_Minor,priority_Trivial,issue_type_Brainstorming,issue_type_Bug,...,issue_type_Improvement,issue_type_New Feature,issue_type_Question,issue_type_Story,issue_type_Sub-task,issue_type_Suitable Name Search,issue_type_Task,issue_type_Technical task,issue_type_Test,issue_type_Wish
0,{'$oid': '5b3b320030a71b06bc6c8ea1'},1.0,17.779244,0,0,1,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
1,{'$oid': '5b3b320030a71b06bc6c8ea1'},1.0,17.779873,0,0,1,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
2,{'$oid': '5b3b320630a71b06bc6cc667'},1.0,17.725177,0,0,1,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
3,{'$oid': '5b3b302130a71b06bc5042e9'},1.0,22.499207,0,0,1,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
4,{'$oid': '5b35273530a71b06bc5391ba'},1.0,2.491320,0,0,1,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0


In [135]:
y_class=df5.status
x_class=df5.drop(columns=['creator_id','timetofix','status'])
X_train, X_test, y_train, y_test = train_test_split(x_class, y_class, test_size=0.3) # 70% training and 30% test

In [106]:
# Entfernen von Außreisern
create_25 = df.describe().T.loc['created_at', '25%']
create_75 = df.describe().T.loc['created_at', '75%']
update_25 = df.describe().T.loc['updated_at', '25%']
update_75 = df.describe().T.loc['updated_at', '75%']

IQR_create = 1.5 * (create_75 - create_25)
IQR_update = 1.5 * (update_75 - update_25)

df = df.query(' created_at >= (@create_25 - @IQR_create) & created_at <= (@create_75 + @IQR_create) ')
df = df.query(' updated_at >= (@update_25 - @IQR_update) & updated_at <= (@update_75 + @IQR_update) ')

In [136]:
print(X_train)

      priority_Blocker  priority_Critical  priority_Major  priority_Minor  \
5646                 0                  0               0               1   
4541                 0                  0               1               0   
1071                 0                  0               1               0   
4644                 0                  0               1               0   
2395                 0                  0               1               0   
...                ...                ...             ...             ...   
670                  0                  0               0               1   
4274                 1                  0               0               0   
726                  0                  0               1               0   
572                  0                  0               1               0   
214                  0                  0               1               0   

      priority_Trivial  issue_type_Brainstorming  issue_type_Bug  \
5646   

In [137]:
clf = tree.DecisionTreeClassifier()
clf = clf.fit(x_class, y_class)
clf = clf.fit(X_train,y_train)
y_pred = clf.predict(X_test)
print("Accuracy:",metrics.accuracy_score(y_test, y_pred))

Accuracy: 0.8408128704487722


In [138]:
from sklearn.ensemble import RandomForestClassifier

#Create a Gaussian Classifier
clrf=RandomForestClassifier(n_estimators=1000)

#Train the model using the training sets y_pred=clf.predict(X_test)
clrf.fit(X_train,y_train)

y_pred=clrf.predict(X_test)
print("Accuracy:",metrics.accuracy_score(y_test, y_pred))

Accuracy: 0.8412362404741744


In [139]:
from sklearn.linear_model import LogisticRegression
logisticRegr = LogisticRegression()
logisticRegr.fit(X_train, y_train)
predictions = logisticRegr.predict(X_test)
print("Accuracy:",metrics.accuracy_score(y_test, predictions))

Accuracy: 0.8416596104995766


In [125]:
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn import metrics

In [130]:
from sklearn import metrics
cm = metrics.confusion_matrix(y_test, predictions)
print(cm)
score = logisticRegr.score(X_test, y_test)
print(score)

[[  27  756]
 [  12 3929]]
0.8374259102455546


In [134]:
plt.figure(figsize=(9,9))
sns.heatmap(cm, annot=True, fmt=".3f", linewidths=.5, square = True, cmap = 'Blues_r')
plt.ylabel('Actual label')
plt.xlabel('Predicted label')
all_sample_title = 'Accuracy Score: {0}'.format(score)
plt.title(all_sample_title, size = 15)


Text(0.5, 1.0, 'Accuracy Score: 0.8374259102455546')

In [49]:
with pd.option_context('display.max_rows', None, 'display.max_columns', None):  # more options can be specified also
    print(df1)

NameError: name 'df1' is not defined